Importing Libraries

In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import Adam, Nadam
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.callbacks import ModelCheckpoint
import imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.metrics import confusion_matrix
import sklearn
import tensorflow as tf
from google.colab import files
import pickle
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import pickle

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# **Random Forest threshold 0.01**

In [ ]:
with open('/content/drive/My Drive/Auto-AI-2019-20/binary training list/CLASSIFICATION FINAL/train_add005.pickle', 'rb') as handle:
    training_dataa = pickle.load(handle)

with open('/content/drive/My Drive/Auto-AI-2019-20/binary training list/CLASSIFICATION FINAL/label_add005.pickle', 'rb') as handle:
    training_labelss = pickle.load(handle)

In [ ]:
# Using SMOTE 
X_resampled, y_resampled = BorderlineSMOTE().fit_resample(training_dataa, training_labelss)
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=0, test_size=0.1)
#x_train, x_test, y_train, y_test = train_test_split(training_dataa, training_labelss, random_state=0, test_size=0.1)



X_train = np.array(x_train)
X_test = np.array(x_test)


#using class weights to balance the data
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights = dict(enumerate(class_weights))

# convert list of labels to binary class matrix
y_train = np_utils.to_categorical(y_train) 
y_test = np_utils.to_categorical(y_test) 


#Reshaping
input_dim = X_train.shape[1]
nb_classes = y_train.shape[1]

# Here's an MLP 
model = Sequential()
model.add(Dense(64, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.35))

model.add(Dense(128, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# model.add(Dense(128))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))



model.add(Dense(nb_classes))
model.add(Activation('softmax'))
opt = Adam(learning_rate=0.0001)


model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('logclass-bin-add05.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
print("Training...")
history = model.fit(X_train, y_train, epochs=250, batch_size=64,
                    validation_split=0.2,
                    callbacks = [es,mc],
                    verbose=1)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Training...
Epoch 1/250
273/278 [============================>.] - ETA: 0s - loss: 1.1545 - accuracy: 0.6632
Epoch 00001: val_loss improved from inf to 0.51072, saving model to logclass-bin-add05.h5
278/278 [==============================] - 1s 3ms/step - loss: 1.1462 - accuracy: 0.6631 - val_loss: 0.5107 - val_accuracy: 0.7610
Epoch 2/250
267/278 [===========================>..] - ETA: 0s - loss: 0.5493 - accuracy: 0.7256
Epoch 00002: val_loss improved from 0.51072 to 0.48298, saving model to logclass-bin-add05.h5
278/278 [==============================] - 1s 3ms/step - loss: 0.5490 - accuracy: 0.7258 - val_loss: 0.4830 - val_accuracy: 0.7686
Epoch 3/250
272/278 [============================>.] - ETA: 0s - loss: 0.5306 - accuracy: 0.7452
Epoch 00003: val_loss improved from 0.48298 to 0.48034, saving model to logclass-bin-add05.h5
278/278 [==============================] - 1s 3ms/step - loss: 0.5316 - accuracy: 0.7450 - val_loss: 0.4803 - val_accuracy: 0.7711
Epoch 4/250
271/278 [=====

In [ ]:
sqr_model = tf.python.keras.models.load_model('logclass-bin-add05.h5')
adam_predictions = sqr_model.predict(X_test, verbose=1)
# prediction and clasification report
y_true, y_pred = [],[]
#classes = le.classes_
for idx, prediction in enumerate(adam_predictions): 
    y_true.append([np.argmax(y_test[idx])])
    y_pred.append([np.argmax(prediction)])
    
print(sklearn.metrics.classification_report(y_pred, y_true))

78/78 [==============================] - 0s 998us/step
              precision    recall  f1-score   support

           0       0.75      0.93      0.83      1003
           1       0.94      0.79      0.86      1464

    accuracy                           0.85      2467
   macro avg       0.85      0.86      0.85      2467
weighted avg       0.86      0.85      0.85      2467



In [ ]:
files.download('logclass-bin-add05.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with open('/content/drive/My Drive/Auto-AI-2019-20/binary training list/CLASSIFICATION FINAL/train_minus005.pickle', 'rb') as handle:
    training_dataa = pickle.load(handle)

with open('/content/drive/My Drive/Auto-AI-2019-20/binary training list/CLASSIFICATION FINAL/label_minus005.pickle', 'rb') as handle:
    training_labelss = pickle.load(handle)

In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

#using SMOTE
X_resampled, y_resampled = BorderlineSMOTE().fit_resample(training_dataa, training_labelss)
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=0, test_size=0.1)
#x_train, x_test, y_train, y_test = train_test_split(training_dataa, training_labelss, random_state=0, test_size=0.2)



X_train = np.array(x_train)
X_test = np.array(x_test)


#using class weights to balance the data
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights = dict(enumerate(class_weights))

#weights = {0:1,1:2.5}
# convert list of labels to binary class matrix
y_train = np_utils.to_categorical(y_train) 
y_test = np_utils.to_categorical(y_test) 


#Reshaping
input_dim = X_train.shape[1]
nb_classes = y_train.shape[1]

# Here's an MLP
model = Sequential()
model.add(Dense(64, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.35))

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.45))

# model.add(Dense(256))
# model.add(Activation('relu'))
# model.add(Dropout(0.45))


# model.add(Dense(128))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))
opt = Adam(learning_rate=0.001)

model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('logclass-bin-minus05.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
print("Training...")
history = model.fit(X_train, y_train, epochs=250, batch_size=64,

                    validation_split=0.2,
                    callbacks = [es,mc],
                    verbose=1)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Training...
Epoch 1/250
379/388 [============================>.] - ETA: 0s - loss: 0.6584 - accuracy: 0.6457
Epoch 00001: val_loss improved from inf to 0.57755, saving model to logclass-bin-minus05.h5
388/388 [==============================] - 1s 3ms/step - loss: 0.6582 - accuracy: 0.6448 - val_loss: 0.5775 - val_accuracy: 0.6520
Epoch 2/250
386/388 [============================>.] - ETA: 0s - loss: 0.6044 - accuracy: 0.6499
Epoch 00002: val_loss did not improve from 0.57755
388/388 [==============================] - 1s 3ms/step - loss: 0.6044 - accuracy: 0.6499 - val_loss: 0.5846 - val_accuracy: 0.6562
Epoch 3/250
374/388 [===========================>..] - ETA: 0s - loss: 0.5914 - accuracy: 0.6670
Epoch 00003: val_loss improved from 0.57755 to 0.55775, saving model to logclass-bin-minus05.h5
388/388 [==============================] - 1s 3ms/step - loss: 0.5907 - accuracy: 0.6688 - val_loss: 0.5577 - val_accuracy: 0.7005
Epoch 4/250
385/388 [============================>.] - ETA: 0s - 

In [ ]:
sqr_model = tf.python.keras.models.load_model('logclass-bin-minus05.h5')
adam_predictions = sqr_model.predict(X_test, verbose=1)
# prediction and clasification report
y_true, y_pred = [],[]
#classes = le.classes_
for idx, prediction in enumerate(adam_predictions): 
    y_true.append([np.argmax(y_test[idx])])
    y_pred.append([np.argmax(prediction)])
    
print(sklearn.metrics.classification_report(y_pred, y_true))

108/108 [==============================] - 0s 839us/step
              precision    recall  f1-score   support

           0       0.77      0.87      0.82      1512
           1       0.89      0.80      0.84      1937

    accuracy                           0.83      3449
   macro avg       0.83      0.83      0.83      3449
weighted avg       0.84      0.83      0.83      3449



In [ ]:
files.download('logclass-bin-minus05.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with open('/content/drive/My Drive/Auto-AI-2019-20/binary training list/CLASSIFICATION FINAL/train_mul005.pickle', 'rb') as handle:
    training_dataa = pickle.load(handle)

with open('/content/drive/My Drive/Auto-AI-2019-20/binary training list/CLASSIFICATION FINAL/label_mul005.pickle', 'rb') as handle:
    training_labelss = pickle.load(handle)

In [ ]:
X_resampled, y_resampled = BorderlineSMOTE().fit_resample(training_dataa, training_labelss)
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=0, test_size=0.1)
#x_train, x_test, y_train, y_test = train_test_split(training_dataa, training_labelss, random_state=0, test_size=0.1)


X_train = np.array(x_train)
X_test = np.array(x_test)


#using class weights to balance the data
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights = dict(enumerate(class_weights))

# convert list of labels to binary class matrix
y_train = np_utils.to_categorical(y_train) 
y_test = np_utils.to_categorical(y_test) 

# weights = {0:1,1:2}

input_dim = X_train.shape[1]
nb_classes = y_train.shape[1]

# Here's an MLP 
model = Sequential()
model.add(Dense(128, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.35))

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# model.add(Dense(256))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))


# model.add(Dense(256))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))
opt = Adam(learning_rate=0.0001)

model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=6)
mc = ModelCheckpoint('logclass-bin-mul05.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
print("Training...")
history = model.fit(X_train, y_train, epochs=250, batch_size=16, 
                    validation_split=0.2,
                    callbacks = [mc,es],
                    verbose=1)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Training...
Epoch 1/250
1617/1641 [============================>.] - ETA: 0s - loss: 0.7729 - accuracy: 0.6890
Epoch 00001: val_loss improved from inf to 0.48244, saving model to logclass-bin-mul05.h5
1641/1641 [==============================] - 3s 2ms/step - loss: 0.7694 - accuracy: 0.6896 - val_loss: 0.4824 - val_accuracy: 0.7899
Epoch 2/250
1631/1641 [============================>.] - ETA: 0s - loss: 0.4943 - accuracy: 0.7740
Epoch 00002: val_loss improved from 0.48244 to 0.43757, saving model to logclass-bin-mul05.h5
1641/1641 [==============================] - 3s 2ms/step - loss: 0.4945 - accuracy: 0.7738 - val_loss: 0.4376 - val_accuracy: 0.8026
Epoch 3/250
1634/1641 [============================>.] - ETA: 0s - loss: 0.4613 - accuracy: 0.7957
Epoch 00003: val_loss improved from 0.43757 to 0.41213, saving model to logclass-bin-mul05.h5
1641/1641 [==============================] - 3s 2ms/step - loss: 0.4608 - accuracy: 0.7960 - val_loss: 0.4121 - val_accuracy: 0.8043
Epoch 4/250
16

In [ ]:
sqr_model = tf.python.keras.models.load_model('logclass-bin-mul05.h5')
adam_predictions = sqr_model.predict(X_test, verbose=1)
# prediction and clasification report
y_true, y_pred = [],[]
#classes = le.classes_
for idx, prediction in enumerate(adam_predictions): 
    y_true.append([np.argmax(y_test[idx])])
    y_pred.append([np.argmax(prediction)])
    
print(sklearn.metrics.classification_report(y_pred, y_true))

114/114 [==============================] - 0s 982us/step
              precision    recall  f1-score   support

           0       0.84      0.98      0.91      1526
           1       0.99      0.86      0.92      2121

    accuracy                           0.91      3647
   macro avg       0.91      0.92      0.91      3647
weighted avg       0.93      0.91      0.92      3647



In [ ]:
files.download('logclass-bin-mul05.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with open('/content/drive/My Drive/Auto-AI-2019-20/New Training Data/binary/005/train_mul005.pickle', 'rb') as handle:
    training_dataa = pickle.load(handle)

with open('/content/drive/My Drive/Auto-AI-2019-20/New Training Data/binary/005/label_mul005.pickle', 'rb') as handle:
    training_labelss = pickle.load(handle)

In [ ]:
np.unique(training_labelss, return_counts=True)

(array([0, 1]), array([10331,  4389]))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(training_dataa, training_labelss, random_state=0, test_size=0.1)


X_train = np.array(x_train)
X_test = np.array(x_test)


#using class weights to balance the data
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights = dict(enumerate(class_weights))

# convert list of labels to binary class matrix
y_train = np_utils.to_categorical(y_train) 
y_test = np_utils.to_categorical(y_test) 

# weights = {0:1,1:2}

input_dim = X_train.shape[1]
nb_classes = y_train.shape[1]

# Here's an MLP 
model = Sequential()
model.add(Dense(32, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.35))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.35))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))



model.add(Dense(nb_classes))
model.add(Activation('softmax'))
opt = Adam(learning_rate=0.0001)

model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=6)
mc = ModelCheckpoint('bin-mul05.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
print("Training...")
history = model.fit(X_train, y_train, nb_epoch=250, batch_size=64, class_weight=class_weights,
                    validation_split=0.2,
                    callbacks = [mc,es],
                    verbose=1)

Training...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 10598 samples, validate on 2650 samples
Epoch 1/250
10598/10598 [==============================] - 1s 76us/step - loss: 0.6924 - accuracy: 0.5448 - val_loss: 0.6914 - val_accuracy: 0.6755

Epoch 00001: val_loss improved from inf to 0.69139, saving model to bin-mul05.h5
Epoch 2/250
10598/10598 [==============================] - 0s 45us/step - loss: 0.6883 - accuracy: 0.5593 - val_loss: 0.6840 - val_accuracy: 0.7091

Epoch 00002: val_loss improved from 0.69139 to 0.68396, saving model to bin-mul05.h5
Epoch 3/250
10598/10598 [==============================] - 0s 44us/step - loss: 0.6771 - accuracy: 0.6110 - val_loss: 0.6660 - val_accuracy: 0.6717

Epoch 00003: val_loss improved from 0.68396 to 0.66602, saving model to bin-mul05.h5
Epoch 4/250
10598/10598 [==============================] - 0s 45us/step - loss: 0.6574 - accuracy: 0.6254 - val_loss: 0.6400 - val_accuracy: 0.6264

Epoch 00004: val_loss improved from 0.66602 to 0.64004, saving model to bin-mul05.h5
Epoch 5/250
10598/1

In [ ]:
sqr_model = tf.python.keras.models.load_model('bin-mul05.h5')
adam_predictions = sqr_model.predict(X_test, verbose=1)
# prediction and clasification report
y_true, y_pred = [],[]
#classes = le.classes_
for idx, prediction in enumerate(adam_predictions): 
    y_true.append([np.argmax(y_test[idx])])
    y_pred.append([np.argmax(prediction)])
    
print(sklearn.metrics.classification_report(y_pred, y_true))

46/46 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.88      0.93      0.91       976
           1       0.85      0.75      0.80       496

    accuracy                           0.87      1472
   macro avg       0.87      0.84      0.85      1472
weighted avg       0.87      0.87      0.87      1472



In [ ]:
files.download('bin-mul05.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with open('/content/drive/My Drive/Auto-AI-2019-20/New Training Data/cond_prob/train_reg_sqr.pickle', 'rb') as handle:
    training_dataa = pickle.load(handle)

with open('/content/drive/My Drive/Auto-AI-2019-20/New Training Data/cond_prob/label_reg_sqr.pickle', 'rb') as handle:
    training_labelss = pickle.load(handle)

In [ ]:
X_resampled, y_resampled = BorderlineSMOTE().fit_resample(training_dataa, training_labelss)
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=0, test_size=0.1)



X_train = np.array(x_train)
X_test = np.array(x_test)


#using class weights to balance the data
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights = dict(enumerate(class_weights))

# convert list of labels to binary class matrix
y_train = np_utils.to_categorical(y_train) 
y_test = np_utils.to_categorical(y_test) 



input_dim = X_train.shape[1]
nb_classes = y_train.shape[1]

# Here's an MLP
model = Sequential()
model.add(Dense(16, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.35))

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# model.add(Dense(128))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))


model.add(Dense(nb_classes))
model.add(Activation('softmax'))
opt = Adam(learning_rate=0.0001)

model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('lnr-cp-sqr01.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
print("Training...")
history = model.fit(X_train, y_train, nb_epoch=250, batch_size=16,
                    validation_split=0.2,
                    callbacks = [mc,es],
                    verbose=1)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Training...
Train on 842 samples, validate on 211 samples
Epoch 1/250
842/842 [==============================] - 0s 275us/step - loss: 0.6950 - accuracy: 0.4941 - val_loss: 0.6979 - val_accuracy: 0.4313

Epoch 00001: val_loss improved from inf to 0.69786, saving model to lnr-cp-sqr01.h5
Epoch 2/250
842/842 [==============================] - 0s 105us/step - loss: 0.6939 - accuracy: 0.5083 - val_loss: 0.6963 - val_accuracy: 0.4550

Epoch 00002: val_loss improved from 0.69786 to 0.69628, saving model to lnr-cp-sqr01.h5
Epoch 3/250
842/842 [==============================] - 0s 102us/step - loss: 0.6916 - accuracy: 0.5249 - val_loss: 0.6953 - val_accuracy: 0.4787

Epoch 00003: val_loss improved from 0.69628 to 0.69527, saving model to lnr-cp-sqr01.h5
Epoch 4/250
842/842 [==============================] - 0s 107us/step - loss: 0.6897 - accuracy: 0.5451 - val_loss: 0.6954 - val_accuracy: 0.4882

Epoch 00004: val_loss did not improve from 0.69527
Epoch 5/250
842/842 [==========================

In [ ]:
sqr_model = tf.python.keras.models.load_model('lnr-cp-sqr01.h5')
adam_predictions = sqr_model.predict(X_test, verbose=1)
# prediction and clasification report
y_true, y_pred = [],[]
#classes = le.classes_

for idx, prediction in enumerate(adam_predictions): 
    y_true.append([np.argmax(y_test[idx])])
    y_pred.append([np.argmax(prediction)])
    
print(sklearn.metrics.classification_report(y_pred, y_true))

4/4 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.87      0.90      0.89        60
           1       0.89      0.86      0.88        57

    accuracy                           0.88       117
   macro avg       0.88      0.88      0.88       117
weighted avg       0.88      0.88      0.88       117



In [ ]:
files.download('lnr-cp-sqr01.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>